# Cymatic Indra - Simulation Results Analysis

Analyze exported simulation data from multiple runs.
- Load JSON metadata and statistics
- Plot coherence evolution over time
- Visualize phase dynamics and manifold geometry
- Compare multiple simulation runs

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up styling
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.facecolor'] = '#0b0d17'
plt.rcParams['axes.facecolor'] = '#1a1d2e'
plt.rcParams['text.color'] = 'white'

print("✓ Libraries imported successfully")

## 2. Load Simulation Results

In [ ]:
# Find all simulation runs
results_dir = Path('results')
runs = sorted([d for d in results_dir.iterdir() if d.is_dir()])

print(f"Found {len(runs)} simulation run(s)\n")
print("Available runs:")
for run in runs:
    print(f"  - {run.name}")

In [ ]:
# Load the most recent run
latest_run = runs[-1] if runs else None

if latest_run:
    print(f"Loading: {latest_run.name}\n")
    
    # Load JSON metadata
    with open(latest_run / 'simulation.json', 'r') as f:
        sim_data = json.load(f)
    
    # Load CSV time-series
    df = pd.read_csv(latest_run / 'timeseries.csv')
    
    # Load config
    with open(latest_run / 'config.json', 'r') as f:
        config = json.load(f)
    
    print(f"Simulation Metadata:")
    print(f"  Agents: {sim_data['metadata']['n_agents']}")
    print(f"  Dimensions: {sim_data['metadata']['dimensions']}")
    print(f"  Steps: {sim_data['metadata']['num_steps']}")
    print(f"  Timestamp: {sim_data['metadata']['timestamp']}")
    print(f"\nDataFrame shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
else:
    print("No simulation runs found. Please run simulation_runner.py first.")

## 3. Statistical Summary

In [ ]:
# Extract statistics
stats = sim_data['statistics']

print("="*60)
print("SIMULATION STATISTICS")
print("="*60)
print(f"Max Coherence:           {stats['max_coherence']:.4f}")
print(f"Avg Coherence:           {stats['avg_coherence']:.4f}")
print(f"Min Coherence:           {stats['min_coherence']:.4f}")
print(f"Coherence Std Dev:       {np.std(stats['coherence_history']):.4f}")
print(f"Crystallization Events:  {stats['crystallization_events']}")
print(f"\nCoherence Range:         [{stats['min_coherence']:.3f}, {stats['max_coherence']:.3f}]")
print("="*60)

## 4. Coherence Evolution Over Time

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot 1: Coherence over time
ax = axes[0]
ax.plot(df['step'], df['coherence'], linewidth=2, color='#00ced1', alpha=0.8)
ax.axhline(y=0.8, color='#39ff14', linestyle='--', label='Crystallization Threshold', alpha=0.5)
ax.axhline(y=stats['avg_coherence'], color='#ff6b6b', linestyle='--', label='Average Coherence', alpha=0.5)
ax.fill_between(df['step'], df['coherence'], alpha=0.2, color='#00ced1')
ax.set_xlabel('Simulation Step', fontsize=12, color='white')
ax.set_ylabel('Coherence', fontsize=12, color='white')
ax.set_title('Global Coherence Evolution', fontsize=14, fontweight='bold', color='white')
ax.legend(loc='best', facecolor='#1a1d2e', edgecolor='white')
ax.grid(True, alpha=0.2)

# Plot 2: Phase dynamics of first oscillator
ax = axes[1]
ax.plot(df['step'], df['phase_0'], linewidth=1.5, color='#a0a0ff', alpha=0.8, label='Phase (Oscillator 0)')
ax.set_xlabel('Simulation Step', fontsize=12, color='white')
ax.set_ylabel('Phase (radians)', fontsize=12, color='white')
ax.set_title('Phase Dynamics (First Oscillator)', fontsize=14, fontweight='bold', color='white')
ax.legend(loc='best', facecolor='#1a1d2e', edgecolor='white')
ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

## 5. Manifold Geometry Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Manifold energy over time
ax = axes[0]
ax.plot(df['step'], df['manifold_energy'], linewidth=2, color='#39ff14', alpha=0.8)
ax.fill_between(df['step'], df['manifold_energy'], alpha=0.2, color='#39ff14')
ax.set_xlabel('Simulation Step', fontsize=12, color='white')
ax.set_ylabel('Manifold Energy (L2 Norm)', fontsize=12, color='white')
ax.set_title('Manifold Geometry Evolution', fontsize=14, fontweight='bold', color='white')
ax.grid(True, alpha=0.2)

# Plot 2: Phase range dynamics
ax = axes[1]
phase_range = df['max_phase'] - df['min_phase']
ax.fill_between(df['step'], df['min_phase'], df['max_phase'], alpha=0.3, color='#a0a0ff', label='Phase Range')
ax.plot(df['step'], df['phase_0'], linewidth=1, color='#ff6b6b', alpha=0.6, label='Phase (Osc 0)')
ax.set_xlabel('Simulation Step', fontsize=12, color='white')
ax.set_ylabel('Phase (radians)', fontsize=12, color='white')
ax.set_title('Phase Space Spread', fontsize=14, fontweight='bold', color='white')
ax.legend(loc='best', facecolor='#1a1d2e', edgecolor='white')
ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

## 6. Crystallization Events

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Cumulative crystallization events
ax = axes[0]
ax.plot(df['step'], df['crystallization_events'], linewidth=2.5, color='#39ff14', marker='o', markersize=4)
ax.fill_between(df['step'], df['crystallization_events'], alpha=0.2, color='#39ff14')
ax.set_xlabel('Simulation Step', fontsize=12, color='white')
ax.set_ylabel('Cumulative Events', fontsize=12, color='white')
ax.set_title('Crystallization Events', fontsize=14, fontweight='bold', color='white')
ax.grid(True, alpha=0.2)

# Plot 2: Coherence distribution
ax = axes[1]
ax.hist(df['coherence'], bins=30, color='#00ced1', alpha=0.7, edgecolor='white')
ax.axvline(stats['avg_coherence'], color='#ff6b6b', linestyle='--', linewidth=2, label='Mean')
ax.axvline(0.8, color='#39ff14', linestyle='--', linewidth=2, label='Threshold')
ax.set_xlabel('Coherence', fontsize=12, color='white')
ax.set_ylabel('Frequency', fontsize=12, color='white')
ax.set_title('Coherence Distribution', fontsize=14, fontweight='bold', color='white')
ax.legend(facecolor='#1a1d2e', edgecolor='white')
ax.grid(True, alpha=0.2, axis='y')

plt.tight_layout()
plt.show()

print(f"\nTotal Crystallization Events: {int(df['crystallization_events'].iloc[-1])}")

## 7. Compare Multiple Runs

In [ ]:
# Load all runs for comparison
all_runs_data = []

for run_dir in runs:
    with open(run_dir / 'simulation.json', 'r') as f:
        sim = json.load(f)
    all_runs_data.append({
        'run': run_dir.name,
        'max_coherence': sim['statistics']['max_coherence'],
        'avg_coherence': sim['statistics']['avg_coherence'],
        'min_coherence': sim['statistics']['min_coherence'],
        'crystallizations': sim['statistics']['crystallization_events']
    })

comparison_df = pd.DataFrame(all_runs_data)
print("\nSimulation Runs Comparison:")
print(comparison_df.to_string(index=False))

In [ ]:
if len(runs) > 1:
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))
    
    # Plot 1: Max coherence comparison
    ax = axes[0, 0]
    ax.bar(range(len(comparison_df)), comparison_df['max_coherence'], color='#00ced1', alpha=0.7, edgecolor='white')
    ax.set_xlabel('Run', fontsize=10, color='white')
    ax.set_ylabel('Max Coherence', fontsize=10, color='white')
    ax.set_title('Maximum Coherence by Run', fontsize=12, fontweight='bold', color='white')
    ax.set_xticks(range(len(comparison_df)))
    ax.set_xticklabels([r.split('_')[-1] for r in comparison_df['run']], rotation=45)
    ax.grid(True, alpha=0.2, axis='y')
    
    # Plot 2: Avg coherence comparison
    ax = axes[0, 1]
    ax.bar(range(len(comparison_df)), comparison_df['avg_coherence'], color='#a0a0ff', alpha=0.7, edgecolor='white')
    ax.set_xlabel('Run', fontsize=10, color='white')
    ax.set_ylabel('Avg Coherence', fontsize=10, color='white')
    ax.set_title('Average Coherence by Run', fontsize=12, fontweight='bold', color='white')
    ax.set_xticks(range(len(comparison_df)))
    ax.set_xticklabels([r.split('_')[-1] for r in comparison_df['run']], rotation=45)
    ax.grid(True, alpha=0.2, axis='y')
    
    # Plot 3: Coherence range
    ax = axes[1, 0]
    ax.errorbar(range(len(comparison_df)), comparison_df['avg_coherence'], 
                yerr=[comparison_df['avg_coherence'] - comparison_df['min_coherence'],
                      comparison_df['max_coherence'] - comparison_df['avg_coherence']],
                fmt='o', color='#39ff14', ecolor='#ff6b6b', capsize=5, markersize=8)
    ax.set_xlabel('Run', fontsize=10, color='white')
    ax.set_ylabel('Coherence', fontsize=10, color='white')
    ax.set_title('Coherence Range by Run', fontsize=12, fontweight='bold', color='white')
    ax.set_xticks(range(len(comparison_df)))
    ax.set_xticklabels([r.split('_')[-1] for r in comparison_df['run']], rotation=45)
    ax.grid(True, alpha=0.2)
    
    # Plot 4: Crystallization events
    ax = axes[1, 1]
    ax.bar(range(len(comparison_df)), comparison_df['crystallizations'], color='#ff6b6b', alpha=0.7, edgecolor='white')
    ax.set_xlabel('Run', fontsize=10, color='white')
    ax.set_ylabel('Events', fontsize=10, color='white')
    ax.set_title('Crystallization Events by Run', fontsize=12, fontweight='bold', color='white')
    ax.set_xticks(range(len(comparison_df)))
    ax.set_xticklabels([r.split('_')[-1] for r in comparison_df['run']], rotation=45)
    ax.grid(True, alpha=0.2, axis='y')
    
    plt.tight_layout()
    plt.show()
else:
    print("Only one run available. Run multiple simulations to compare.")

## 8. Export Summary Report

In [ ]:
# Generate summary report
report = f"""
CYMATIC INDRA - SIMULATION ANALYSIS REPORT
{'='*60}

Run: {latest_run.name}
Timestamp: {sim_data['metadata']['timestamp']}

SIMULATION PARAMETERS:
  Agents: {sim_data['metadata']['n_agents']}
  Dimensions: {sim_data['metadata']['dimensions']}
  Steps: {sim_data['metadata']['num_steps']}
  Engine Config: {json.dumps(config['engine_config'], indent=2)}

RESULTS:
  Max Coherence: {stats['max_coherence']:.4f}
  Avg Coherence: {stats['avg_coherence']:.4f}
  Min Coherence: {stats['min_coherence']:.4f}
  Std Dev: {np.std(stats['coherence_history']):.4f}
  Crystallization Events: {stats['crystallization_events']}
  
KEY FINDINGS:
  - System achieved {stats['max_coherence']:.1%} coherence
  - Average synchronization: {stats['avg_coherence']:.1%}
  - Coherence variability: {np.std(stats['coherence_history']):.4f}
  - Crystallization threshold: 0.8
  - Events above threshold: {stats['crystallization_events']}

{'='*60}
"""

print(report)

# Save report
report_path = latest_run / 'analysis_report.txt'
with open(report_path, 'w') as f:
    f.write(report)

print(f"✓ Report saved to: {report_path}")